In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
'''Definir periodo de análisis: fecha de inicio y fin. 
   Comprendido entre [1981-01-01, 2021-11-30]'''
fecha_inicio = '1981-01-01'
fecha_fin = '2020-12-31'


'''devuelve los datos de una variable'''
def variable_dato(data, var):
    v = data.variables[var][:]
    return v

def creardiario(cad):
    anio = cad[2].split('.')
    md_i = '-01-01'
    if anio[0] == '2021':
        md_f = '-11-30'
    else:
        md_f = '-12-31'
    d1 = anio[0]+md_i
    d2 = anio[0]+md_f
    data = creardf(d1, d2, 'D')
    return data

'''crear un DF vacío en un rango de fechas'''
def creardf(f_ini = '1981-01-01', f_fin = '2021-11-30', fr = "MS"):
    dat = pd.DataFrame(index = rango_fecha(f_ini, f_fin, fr))
    return dat

'''devuelve un rango de fechas mensual (default: <1981/01/01 - 2021/11/01, mensual>)'''
def rango_fecha(f_ini = '1981-01-01', f_fin = '2021-11-30', f = 'MS'):
    f_rango = pd.date_range(start=f_ini, end=f_fin, freq=f)
    return f_rango.date

'''guarda un archivo <data> con un nombre <nom>'''
def guardar(data, nom, path='../datos/por_variables/'):
    pth = path + nom
    data.to_csv(pth, index = False)

## Datos mensuales a partir de archivo mensual Chirps NC

In [3]:
'''Cargar datos'''
data = Dataset(r'../datos/formato5_chirps/chirps-v2.0.monthly.altiplano.nc')

In [4]:
print(data.variables.keys())

dict_keys(['latitude', 'longitude', 'precip', 'time'])


In [5]:
lat = variable_dato(data,'latitude')
lon = variable_dato(data,'longitude')
time = variable_dato(data,'time')
pp = variable_dato(data,'precip')

In [6]:
'''Leer archivo con las estaciones y coordenadas de referencia'''
df1 = pd.read_csv(r'../datos/formato5_chirps/stations.txt', header =None)

In [7]:
dfpp = creardf()
nom_col = []

for i in range(df1.shape[0]):
    df= pd.DataFrame(columns = ['precipitacion','lon_o','lat_o','lon_n','lat_n','estacion'],index=rango_fecha())
    sq_dif_lat = (lat - df1.iloc[i,2])**2
    sq_dif_lon = (lon - df1.iloc[i,1])**2
    min_index_lat=sq_dif_lat.argmin()
    min_index_lon=sq_dif_lon.argmin()
    dt = np.arange(0,data.variables['time'].size)
    #dt=range(dfpp.shape[0])
    for time_index in dt:
        df.iloc[time_index,1] = df1.iloc[i,1]
        df.iloc[time_index,2] = df1.iloc[i,2]
        df.iloc[time_index,0] = (pp[time_index,min_index_lat,min_index_lon])
        df.iloc[time_index,3] = lon[min_index_lon]
        df.iloc[time_index,4] = lat[min_index_lat]
        df.iloc[time_index,5] = df1.iloc[i,4]
    
    nombre = '../datos/formato5_chirps/chirps_mensual_'+df1.iloc[i,0].astype(str)+'.csv'    
    df['precipitacion'] = df['precipitacion'].astype(float)
    df['precipitacion'] = df['precipitacion'].round(2)
    
    dfpp = pd.concat([dfpp, df['precipitacion']], axis=1)
    nom_col.append(str(df1.iloc[i,0]))
        
    df = df.reset_index()
    df.rename(columns={'index':'fecha'}, inplace=True)
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df[(df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)]
    df.to_csv(nombre,header=False, index = None)

    
    

dfpp.columns = nom_col
dfpp = dfpp.reset_index()
dfpp.rename(columns={'index':'fecha'},inplace=True)
dfpp['fecha'] = pd.to_datetime(dfpp['fecha'])
dfpp = dfpp[(dfpp['fecha'] >= fecha_inicio) & (dfpp['fecha'] <= fecha_fin)]
guardar(dfpp, 'chirps_mes_pp.csv')
#Estructura del dataframe generado con las latitudes y longitudes más cercanas
df

,fecha,precipitacion,lon_o,lat_o,lon_n,lat_n,estacion
0,1981-01-01,81.01,-69.25,-17.533,-69.225006,-17.525002,avaroa
1,1981-02-01,70.56,-69.25,-17.533,-69.225006,-17.525002,avaroa
2,1981-03-01,37.13,-69.25,-17.533,-69.225006,-17.525002,avaroa
3,1981-04-01,10.17,-69.25,-17.533,-69.225006,-17.525002,avaroa
4,1981-05-01,1.68,-69.25,-17.533,-69.225006,-17.525002,avaroa
...,...,...,...,...,...,...,...
475,2020-08-01,4.53,-69.25,-17.533,-69.225006,-17.525002,avaroa
476,2020-09-01,8.87,-69.25,-17.533,-69.225006,-17.525002,avaroa
477,2020-10-01,7.18,-69.25,-17.533,-69.225006,-17.525002,avaroa
478,2020-11-01,7.38,-69.25,-17.533,-69.225006,-17.525002,avaroa


## Datos diarios a partir de archivos anuales con mediciones diarias

In [8]:
'''Leer archivo con las estaciones y coordenadas de referencia
   y el archivo con los nombres de los archivos diarios'''
dflista = pd.read_csv(r'../datos/formato5_chirps/stations.txt', header =None)
dfanio = pd.read_csv(r'../datos/formato5_chirps/datos_anio.csv', header =None)

In [9]:
pp_diario = pd.DataFrame()

for anio in range(dfanio.shape[0]):
    nom_col = []
    diario = creardiario(dfanio.loc[anio,0].split('_'))
    dfpp = diario
    diario = diario.reindex(columns = ['precipitacion','lon_o','lat_o','lon_n','lat_n','estacion']) 
    p = r'../datos/formato5_chirps/' + dfanio.loc[anio,0]
    data = Dataset(p)
    lat = variable_dato(data,'latitude')
    lon = variable_dato(data,'longitude')
    time = variable_dato(data,'time')
    pp = variable_dato(data,'precip')
    for i in range(dflista.shape[0]):
        sq_dif_lat = (lat - dflista.iloc[i,2])**2
        sq_dif_lon = (lon - dflista.iloc[i,1])**2
        min_index_lat=sq_dif_lat.argmin()
        min_index_lon=sq_dif_lon.argmin()
        dt = np.arange(0,data.variables['time'].size)  
        for time_index in dt:
            diario.iloc[time_index,1] = dflista.iloc[i,1]
            diario.iloc[time_index,2] = dflista.iloc[i,2]
            diario.iloc[time_index,0] = (pp[time_index,min_index_lat,min_index_lon])
            diario.iloc[time_index,3] = lon[min_index_lon]
            diario.iloc[time_index,4] = lat[min_index_lat]
            diario.iloc[time_index,5] = dflista.iloc[i,4]
        nombre = '../datos/formato_csv_corregido/chirps_'+dflista.iloc[i,0].astype(str)+'_'+dflista.iloc[i,4]+'.csv'
        diario['precipitacion']=diario['precipitacion'].astype(float)
        diario['precipitacion']=diario['precipitacion'].round(2)
        if anio == 0:
            diario.to_csv(nombre,header=False )
        else:
            station = pd.read_csv(nombre, header=None, names=['precipitacion','lon_o','lat_o','lon_n','lat_n','estacion'])
            station = pd.concat ([station, diario])
            station.to_csv(nombre,header=False )
        dfpp = pd.concat([dfpp, diario['precipitacion']], axis=1)
        nom_col.append(str(dflista.iloc[i,0]))
    pp_diario = pd.concat([pp_diario, dfpp], axis=0)

pp_diario.columns = nom_col
pp_diario = pp_diario.reset_index()
pp_diario.rename(columns={'index':'fecha'},inplace=True)
pp_diario['fecha'] = pd.to_datetime(pp_diario['fecha'])
pp_diario = pp_diario[(pp_diario['fecha'] >= fecha_inicio) & (pp_diario['fecha'] <= fecha_fin)]
guardar(pp_diario, 'chirps_diario_pp.csv')

for i in range(dflista.shape[0]):
    nombre = '../datos/formato_csv_corregido/chirps_'+dflista.iloc[i,0].astype(str)+'_'+dflista.iloc[i,4]+'.csv'
    estacion = pd.read_csv(nombre, header=None, usecols = [i for i in range(2)])
    estacion.columns = ['fecha', 'pp']
    estacion['fecha'] = pd.to_datetime(estacion['fecha'])
    estacion = estacion[(estacion['fecha'] >= fecha_inicio) & (estacion['fecha'] <= fecha_fin)]
    estacion.to_csv(nombre, index=False)
    